### INMET Database Maker
#### About:
The **National Institute of Meteorology** of Brazil provides weather station data free of charge via the website: https://bdmep.inmet.gov.br/

The data comes in *.csv* format, containing a header and a table.
I developed this script to read each of these files separately, extract the necessary information from the header, assigning them to new columns and separating only the most relevant columns to create a single weather database.

Any doubts, contact me:

**Email**: josehenriqueroveda@usp.br

**LinkedIn**: https://linkedin.com/in/jhroveda

*Attribution Noncommercial (BY-NC)*

In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import create_engine
import numpy as np
import pyodbc
import os
import re
import datetime
from dateutil import parser
import time

In [ ]:
# Database connection
server = 'SERVER_NAME' # to specify an alternate port
driver = 'DRIVER'
database = 'DATABASE_NAME' 
username = 'USER_NAME' 
password = 'PASSWORD'
database_connection = f'mssql://{username}:{password}@{server}/{database}?driver={driver}' # e.g. SQL Server
engine = create_engine(database_connection)

In [ ]:
# Read the folder that contains subfolders by year
years_dir = os.listdir('YOUR_PATH')

In [ ]:
# Get weather station name
def station_name(garbage_string):
    splitted = garbage_string.split(maxsplit=1)
    if(len(splitted) == 1):
        station = splitted[0][9:]
    else:
        station = splitted[0][9:] + ' ' + splitted[1]
    return station

In [ ]:
# Get the week of the year
def year_week(ymd_date):
    return int(datetime.date(ymd_date.year, ymd_date.month, ymd_date.day).strftime('%W'))

In [ ]:
# Prepare and convert String to float
def str_to_float(str_value):
    str_value = str_value.replace(',', 'v').replace('v', '.')
    return float(str_value)

In [ ]:
# Prepare the data and load to the database
def load_weather_station(directory, file_name):
    # Starts the function execution time count
    tic = time.perf_counter()
    
    # Opens the file
    path = r"DATA_PATH" + directory + '/' + file_name
    with open(path, 'r+') as f:
        # get the station location information
        location_block = [next(f) for x in range(8)]
        # Separate the information that will be used in the new columns
        # Numeric variables
        lat = location_block[4]
        lng = location_block[5]
        alt = location_block[6]
        
        # Read the file
        lines = f.readlines()   
        # move the pointer to the beginning of the file
        f.seek(0)
        # truncate the file
        f.truncate()
        # overwrite the structured file
        f.writelines(lines)
    
    uf = re.search(':;{1,}(.+?)\W', location_block[1]).group(1)
    estacao = station_name(location_block[2])
    latitude = re.search(':;{1,}(.+?)(;|\\n)', location_block[4]).group(1)
    longitude = re.search(':;{1,}(.+?)(;|\\n)', location_block[5]).group(1)
    altitude = re.search(':;{1,}(.+?)(;|\\n)', location_block[6]).group(1)
    
    # Open the rest of the file as a DataFrame
    df = pd.read_csv(path, sep=';',encoding="latin-1", usecols=['DATA (YYYY-MM-DD)', 'HORA (UTC)', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)', 'RADIACAO GLOBAL (KJ/m²)', 
                                                                'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)', 'UMIDADE RELATIVA DO AR, HORARIA (%)', 
                                                                'VENTO, VELOCIDADE HORARIA (m/s)'])
    
    # rename the remaining columns
    df.columns = ['data_medicao', 'hora_utc', 'precipitacao_total', 'radiacao_global', 'temperatura_ar', 'umidade_relativa', 'velocidade_vento']
    
    # create new columns with station information
    df['uf'] = uf
    df['estacao'] = estacao
    df['latitude'] = latitude
    df['longitude'] = longitude
    df['altitude'] = altitude
    
    # Create the Year, Month, Day columns
    df['data_medicao'] = df['data_medicao'].apply(lambda x: x.replace('/', '-'))
    df['data_medicao'] = df['data_medicao'].apply(lambda x: parser.parse(x))
    df['ano'] = df['data_medicao'].apply(lambda x: x.year)
    df['mes'] = df['data_medicao'].apply(lambda x: x.month)
    df['dia'] = df['data_medicao'].apply(lambda x: x.day)
    
    
    # Convert numeric columns to correct types
    df['precipitacao_total'] = df['precipitacao_total'].apply(lambda x: str_to_float(str(x)))
    df['radiacao_global'] = df['radiacao_global'].apply(lambda x: str_to_float(str(x)))
    df['temperatura_ar'] = df['temperatura_ar'].apply(lambda x: str_to_float(str(x)))
    df['umidade_relativa'] = pd.to_numeric(df['umidade_relativa'], errors='coerce')
    df['altitude'] = pd.to_numeric(df['altitude'], errors='coerce')
    df['velocidade_vento'] = df['velocidade_vento'].apply(lambda x: str_to_float(str(x)))
    df['ano'] = pd.to_numeric(df['ano'], errors='coerce')
    df['mes'] = pd.to_numeric(df['mes'], errors='coerce')
    df['dia'] = pd.to_numeric(df['dia'], errors='coerce')
    
    # Create the Day of the Year column
    df['dia_ano'] = df['data_medicao'].apply(lambda x: x.timetuple().tm_yday)
    df['dia_ano'] = pd.to_numeric(df['dia_ano'], errors='coerce')
    
    # Create the Week column
    df['semana'] = df['data_medicao'].apply(lambda x: year_week(x))
    
    # Reorder the columns
    df = df[['data_medicao', 'dia', 'mes', 'ano', 'dia_ano', 'semana', 'hora_utc', 'precipitacao_total', 
             'radiacao_global', 'temperatura_ar', 'umidade_relativa', 'velocidade_vento',
             'estacao', 'uf', 'latitude', 'longitude', 'altitude']]
    
    # Remove dirty data from base
    df = df.replace([-9999],None)
    
    # Insert into database
    df.to_sql('WEATHER_TABLE_NAME', con=engine, schema='YOUR_SCHEMA', if_exists='append', index=False)
    
    # Function execution time
    toc = time.perf_counter()
    print(f'File: {file_name} loaded to database. Execution time: {toc - tic:0.4f} seconds')

In [ ]:
# Start measuring the execution time
start_time = time.perf_counter()

# Load the data for each file in each year folder
for year in years_dir:
    files_dir = os.listdir(r'DATA_PATH' + year + '/')
    for file_name in files_dir:
        load_weather_station(year, file_name)
        
# Print the measure of execution time
end_time = time.perf_counter()
print(f'Weather Stations data loaded to database. Total execution time: {end_time - start_time:0.4f} seconds')